In [ ]:
import pandas as pd
import json 
from fastparquet import ParquetFile
import jsonlines
import os

datasets_path = '/data/WorkSpace/openbmb/dataset/light_beir_eval_c'
# read all dataset dirs
ls = os.listdir(datasets_path)
for dataset in ls:
    dataset_path = os.path.join(datasets_path, dataset,'data')
    ls_on_dataset_path = os.listdir(dataset_path)
    for fl in ls_on_dataset_path:
        fl_path = os.path.join(dataset_path, fl)
        if fl.endswith('.parquet'):
            pf_path = fl_path
            pf = ParquetFile(pf_path)
            df = pf.to_pandas()
            df.rename(columns={'id':'_id'}, inplace=True)
            # to jsonl
            prefix = fl.split('-')[0]
            df.to_json(dataset_path +'/' + prefix + '.jsonl'
                       , orient='records',index=False, lines=True,force_ascii=False)
    tsv_path = os.path.join(datasets_path,dataset, dataset+'-qrels', 'data')
    ls_on_tsv_path = os.listdir(tsv_path)
    for fl in ls_on_tsv_path:
        if fl.endswith('.parquet'):
            fl_path = os.path.join(tsv_path, fl)
            pf = ParquetFile(fl_path)
            df = pf.to_pandas()
            df.rename(columns={'qid':'query-id','pid':'corpus-id'}, inplace=True)
            prefix = fl.split('-')[0]
            df.to_csv(tsv_path +'/' +'test.tsv'
                       , sep='\t', index=False)

In [12]:
import pandas as pd
import json 
import jsonlines
import os

datasets_path = '/data/WorkSpace/openbmb/dataset/light_beir_eval_c_test'
# read all dataset dirs
ls = os.listdir(datasets_path)
for dataset in ls:
    corpus_path = os.path.join(datasets_path, dataset, 'corpus.jsonl')
    query_path = os.path.join(datasets_path, dataset, 'queries.jsonl')
    tsv_path = os.path.join(datasets_path, dataset, 'qrels', 'test.tsv')
    df = pd.read_csv(tsv_path, sep='\t', header=None, names=['query_id', 'corpus_id', 'label'])
    # find if query and corpus have same id
    
    df['are_columns_equal'] = df['query_id'] == df['corpus_id']
    print(df['are_columns_equal'].sum(),line)
    
    q_ids = set()
    for line in jsonlines.open(query_path):
        q_ids.add(line['_id'])
    p_ids = set()
    for line in jsonlines.open(corpus_path):
        p_ids.add(line['_id'])
    
    print(len(q_ids & p_ids), dataset)
    if len(q_ids & p_ids):
        and_id = list(q_ids & p_ids)[0]
        with open(os.path.join(datasets_path, dataset, 'queries.jsonl'), 'r') as f:
            for line in f:
                line = json.loads(line)
                if line['_id'] == and_id:
                    print(line)
                    break
        with open(os.path.join(datasets_path, dataset, 'corpus.jsonl'), 'r') as f:
            for line in f:
                line = json.loads(line)
                if line['_id'] == and_id:
                    print(line)
                    break
        

0
0
0
0
0
